In [ ]:
#imports

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '/MicrowellProcessor/processingpack')

from copy import deepcopy
from pathlib import Path

import cv2
import numpy as np
import skimage
from skimage import measure
import pandas as pd
import matplotlib.pyplot as pl

import ast
import operator

import org_extension
import chip
import experiment as exp
import chipcollections as collections
from org_extension import *

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
#use ImageJ to to open stitched image from first timepoint
#hover over center of microwells to set outside corner positions - (TL, TR, BL, BR)

oc = ((1285,508), (8169,670), (995,9807), (7934,9974))


In [ ]:
#set parameter for microwell-to-microwell spacing between adjacent subarrays

#10X imaging parameters
#100um
org_extension.INTRA_TILE_SPACING = 192

#200um
# org_extension.INTRA_TILE_SPACING = 305

In [ ]:
#set parameters for rotating stitched image and extracting subarrays
from datetime import datetime

#number of complete subarrays in stitched image
subarray_dims = (3,4) #col,row

#microwell dimensions of subarrays
#100um
tile_dims = (20, 20)
#200um
# tile_dims = (10, 10)

#don't need to change these, they don't impact processing
channel = '2bf'
exposure = 10

#change to reflect the macrowell you're processing
#well A1 = 1, A2 = 2, B1 = 5, etc.
well_id = 'A1'
well =1

#date of processing (year, month, day)
date = datetime(2022, 6, 22)

#list of all timepoints in timecourse
for timepoint in ('20200114_193820', '20200114_222045', '20200115_010311', '20200115_034536', '20200115_062755', '20200115_091009', '20200115_115222', '20200115_143408', '20200115_171549', '20200115_195743', '20200115_223926', '20200116_012108', '20200116_040253', '20200116_064442', '20200116_092622', '20200116_120814', '20200116_145024', '20200116_173236', '20200116_201419', '20200116_225611', '20200117_013808', '20200117_042004', '20200117_070202', '20200117_094409', '20200117_122608', '20200117_150806', '20200117_174958', '20200117_203158', '20200117_231358', '20200118_015600', '20200118_043802', '20200118_072002', '20200118_100203', '20200118_124405', '20200118_152607', '20200118_180806', '20200118_205009', '20200118_233217', '20200119_021423', '20200119_045620', '20200119_073818', '20200119_102016', '20200119_130213', '20200119_154406', '20200119_182602', '20200119_210801', '20200119_235000', '20200120_023156', '20200120_051354', '20200120_075554', '20200120_103756'):
    #change srcHandle and targetHandle to match file location for stitched images
    srcHandle = Path('/scratch/users/aadams0/raw_imaging_data/new/P53_ARID1A_20200114/{0}/{1}-{0}_timecourse/2bf/StitchedImages/StitchedImg_1_2bf_0.tif'.format(well_id, timepoint))
    targetHandle = Path('/scratch/users/aadams0/raw_imaging_data/new/P53_ARID1A_20200114/{0}/{1}-{0}_timecourse/2bf/StitchedImages/RotatedImg_1_2bf_0.tif'.format(well_id, timepoint))
    rot_img = readAndRotateImg(srcHandle, oc, subarray_dims, targetHandle = targetHandle)
    rot_corners = transformCorners(readTiff(srcHandle), rot_img, oc)
    divisions = getPartitions(rot_corners, subarray_dims)
     #change root and outRoot below to match desired location for processed data
    root = '/scratch/users/aadams0/processed_data/Nikon/new2/P53_ARID1A_20200114/{0}/2bf/'.format(well_id)
    description = 'timepoint_{}'.format(timepoint)
    operator = 'AS'
    #change stampWidth depending on 100um vs 200um microwells
    #100um - 10X
    chip.ChipImage.stampWidth = 115
    #200um - 10X
    #     chip.ChipImage.stampWidth = 225
    #200um - 20X
    #     chip.ChipImage.stampWidth = 465
    e = exp.Experiment(description, root, operator) #establish experiment
    pinlist_path = '/home/users/aadams0/microwell_code/Pinlist/20x20_pinlist.csv'
    pinlist = e.read_pinlist(pinlist_path)#establish pinlist
    arrayRepo = processTiles(str(targetHandle), e, pinlist, divisions, tile_dims, subarray_dims, channel, exposure, date = date, well = well, desc = e.info)
    outRoot = Path('/scratch/users/aadams0/processed_data/Nikon/new/P53_ARID1A_20200114/{0}/2bf/'.format(well_id)) #Folder to write summary image and CSV

    ar = arrayRepo[0, 0]
    hashStr = generateWellIdentifier(ar)
    summary_imgPath = outRoot.joinpath('{}.tif'.format(hashStr))
    summary_csvPath = outRoot.joinpath('{}.csv'.format(hashStr))

    summaryDF = summarizeArrayRepo(arrayRepo)
    summaryDF.to_csv(summary_csvPath)
    writeSubArraySummaryImg(arrayRepo, summary_imgPath)